# Import Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
#keyword embedding
import io
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # warnings for tf
import re
import shutil
import string

#pip install tensorflow
import tensorflow as tf

from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

2023-12-01 10:54:52.812600: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 10:54:52.812852: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 10:54:52.827475: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
!pwd

/home/zoetustain/code/zulu-tango/news_and_echo_bubbles/notebooks


In [8]:
#import right-wing news dataset
brainded_right = pd.read_csv('/home/zoetustain/code/zulu-tango/news_and_echo_bubbles/raw_data/braindedright.csv')
#convert date+time column into separate columns
brainded_right[['pdate','time']] = brainded_right['pdate'].str.split(' ', n=1, expand=True)
brainded_right['pdate'] = pd.to_datetime(brainded_right['pdate'])

# we only want relatively recent news
mask = brainded_right['pdate'] > '2020-01-01'
brainded_right = brainded_right[mask].reset_index()

# import left-wing news dataset
brainded_left = pd.read_csv('/home/zoetustain/code/zulu-tango/news_and_echo_bubbles/raw_data/braindedleft.csv')
brainded_left[['pdate','time']] = brainded_left['pdate'].str.split(' ', n=1, expand=True)
brainded_left['pdate'] = pd.to_datetime(brainded_left['pdate'])

mask = brainded_left['pdate'] > '2020-01-01'
brainded_left = brainded_left[mask].reset_index()

# drop empty rows
brainded_right = brainded_right.dropna().reset_index()
brainded_left = brainded_left.dropna().reset_index()

In [9]:
brainded_left.drop(columns={'level_0','index','Unnamed: 0'},inplace=True)
brainded_right.drop(columns={'level_0','index','Unnamed: 0'},inplace=True)

# Prepare data

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
#add classification column to data
brainded_left['classifier'] = 0
brainded_right['classifier'] = 1

In [23]:
data_left = brainded_left[['text','classifier']]
data_right = brainded_right[['text','classifier']]
data_combined = data_left.merge(data_right,how='outer')

In [24]:
processed_data = pd.read_csv('/home/zoetustain/code/zulu-tango/news_and_echo_bubbles/raw_data/cleaned.csv')

In [25]:
X = processed_data['pre_process_text']
y = processed_data.classifier

In [26]:
X.shape, y.shape

((2691,), (2691,))

In [27]:
X

0       msnbc host mehdi hasan went after rep  dan cre...
1       rep  dan crenshaw s  r texas  stock purchase h...
2       sen  ed markey is the massachusetts democrat w...
3       on valentine s day  the major utility that sup...
4       on valentine’s day  the major utility that sup...
                              ...                        
2686    conservatives were bracing for the defection o...
2687    i rise today to offer a few thoughts about the...
2688     mj getty images   i’m still not sure which si...
2689    much ink has been spilled about the supreme co...
2690    we conservatives now find ourselves trying to ...
Name: pre_process_text, Length: 2691, dtype: object

In [28]:
y

0       0
1       0
2       0
3       0
4       0
       ..
2686    1
2687    1
2688    1
2689    1
2690    1
Name: classifier, Length: 2691, dtype: int64

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0,shuffle=True)

# Text Vectorisation

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [13]:
# needs to be on processed data - the vectorisation below has numbers in it

#run grid search on tokeniser

In [22]:
tfidf= TfidfVectorizer()
tfidf.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': None,
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

In [201]:
params = {'min_df': (1, 0.001, 0.01),
          'max_df':(0.95,0.98,1),
          'ngram_range':((1,1), (1, 2)), 
          'max_features': (5000, 10000, 50000)
         }          

In [202]:
gs = GridSearchCV(tfidf,params,cv=5,verbose=1,n_jobs=10,scoring='accuracy')

In [203]:
gs.fit(X_train,y_train)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


/home/zoetustain/.pyenv/versions/3.10.6/envs/echo_news_bubbles/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/zoetustain/.pyenv/versions/3.10.6/envs/echo_news_bubbles/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/zoetustain/.pyenv/versions/3.10.6/envs/echo_news_bubbles/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/home/zoetustain/.pyenv/versions/3.10.6/envs/echo_news_bubbles/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 353, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/home/zoetustain/.pyenv/versions/3.10.6/envs/echo_news_bu

GridSearchCV(cv=5, estimator=TfidfVectorizer(), n_jobs=10,
             param_grid={'max_df': (0.95, 0.98, 1),
                         'max_features': (5000, 10000, 50000),
                         'min_df': (1, 0.001, 0.01),
                         'ngram_range': ((1, 1), (1, 2))},
             scoring='accuracy', verbose=1)

In [ ]:
gs.best

In [17]:
##BARNEY
tf_idf_vectorizer = TfidfVectorizer()

In [18]:
# Training it on the texts
vectorised_words = pd.DataFrame(tf_idf_vectorizer.fit_transform(X.text).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())

vectorised_words

,00,000,0000,00004,0001177,0002,00032597,00075,00088618,0009,...,ﬁlled,ﬁnd,ﬁngers,ﬁre,ﬁred,ﬁres,ﬁring,ﬁrst,ﬂat,ﬂoated
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.066416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.013904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.013904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2876,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2877,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2878,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2879,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
X_train, X_test, y_train, y_test = train_test_split(vectorised_words,y,test_size=0.2)

# Gradient Boosting Classifier

In [33]:
from sklearn.ensemble import GradientBoostingClassifier

In [21]:
clf = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1,max_depth=2,random_state=0)

In [24]:
clf.fit(X_train,y_train)

GradientBoostingClassifier(max_depth=2, n_estimators=1000, random_state=0)

In [25]:
clf.score(X_test, y_test)

0.8526863084922011

# Random Forest Classification

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [19]:
X, y = make_classification(n_samples=1000, n_features=24000,
                            n_informative=2, n_redundant=0,
                            random_state=0, shuffle=False)

In [20]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [21]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [22]:
clf.score(X_test,y_test)

0.5545927209705372

# Optimise classification parameters

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [34]:
gbc = GradientBoostingClassifier(criterion='friedman_mse',
 learning_rate=0.1, max_depth=7,
 max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
 min_samples_split=0.5, min_weight_fraction_leaf=0.0,
 n_estimators=100, random_state=0,
 subsample=1.0, verbose=0, warm_start=False)

In [35]:
pipeline = Pipeline([
            ('count', CountVectorizer(stop_words='english')),
            ('model', gbc)
])

In [36]:
pipeline

Pipeline(steps=[('count', CountVectorizer(stop_words='english')),
                ('model',
                 GradientBoostingClassifier(max_depth=7, min_samples_split=0.5,
                                            random_state=0))])

In [37]:
pipeline.get_params()

{'memory': None,
 'steps': [('count', CountVectorizer(stop_words='english')),
  ('model',
   GradientBoostingClassifier(max_depth=7, min_samples_split=0.5, random_state=0))],
 'verbose': False,
 'count': CountVectorizer(stop_words='english'),
 'model': GradientBoostingClassifier(max_depth=7, min_samples_split=0.5, random_state=0),
 'count__analyzer': 'word',
 'count__binary': False,
 'count__decode_error': 'strict',
 'count__dtype': numpy.int64,
 'count__encoding': 'utf-8',
 'count__input': 'content',
 'count__lowercase': True,
 'count__max_df': 1.0,
 'count__max_features': None,
 'count__min_df': 1,
 'count__ngram_range': (1, 1),
 'count__preprocessor': None,
 'count__stop_words': 'english',
 'count__strip_accents': None,
 'count__token_pattern': '(?u)\\b\\w\\w+\\b',
 'count__tokenizer': None,
 'count__vocabulary': None,
 'model__ccp_alpha': 0.0,
 'model__criterion': 'friedman_mse',
 'model__init': None,
 'model__learning_rate': 0.1,
 'model__loss': 'log_loss',
 'model__max_depth': 7,

In [38]:
parameters = {
        'count__max_features': (5_000,10_000,30_000),
        'count__max_df': (0.95,0.98,1),
        'count__min_df': (0.001,0.01,0.1),
        'count__ngram_range': ((1, 1),(1,2))
        # 'tfidf__max_features': (5_000, 10_000, 30_000),
        # 'tfidf__max_df': (1,5,10),
        # 'tfidf__min_df': (1,0.1,0.001),
        # 'tfidf__ngram_range':((1,1), (1, 2), (1,3))
    }

In [39]:
# grid search
gs = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs=8, verbose=1)

In [40]:
gs.fit(X_train,y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


/home/zoetustain/.pyenv/versions/lewagon/envs/news_and_echo_bubbles/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
54 fits failed out of a total of 162.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "/home/zoetustain/.pyenv/versions/lewagon/envs/news_and_echo_bubbles/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/zoetustain/.pyenv/versions/lewagon/envs/news_and_echo_bubbles/lib/python3.10/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('count',
                                        CountVectorizer(stop_words='english')),
                                       ('model',
                                        GradientBoostingClassifier(max_depth=7,
                                                                   min_samples_split=0.5,
                                                                   random_state=0))]),
             n_jobs=8,
             param_grid={'count__max_df': (0.95, 0.98, 1),
                         'count__max_features': (5000, 10000, 30000),
                         'count__min_df': (0.001, 0.01, 0.1),
                         'count__ngram_range': ((1, 1), (1, 2))},
             verbose=1)

In [232]:
# best estimator to fit
gs.best_params_

{'count__max_df': 0.95,
 'count__max_features': 30000,
 'count__min_df': 1e-05,
 'count__ngram_range': (1, 2)}

In [41]:
cv_model = CountVectorizer(max_df= 0.98, max_features= 20_000, min_df= 0.01, ngram_range= (1, 2),stop_words='english')


In [43]:
import nltk
#nltk.download('wordnet')

In [45]:

from nltk.stem import WordNetLemmatizer

def lemma(text):
  noun_lemmatized = [                 
    WordNetLemmatizer().lemmatize(word, pos = "n") # n --> nouns
    for word in text.split()
    ]
  return ' '.join(noun_lemmatized)

In [46]:
processed_data['lemmatized'] = processed_data['pre_process_text'].apply(lemma)
X = processed_data['lemmatized']

In [47]:
X_train, y_train, X_text, y_test = train_test_split(X,y,test_size=0.2,random_state=0,shuffle=True)

In [48]:
word_count_vector=cv_model.fit_transform(X_train)

In [49]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [50]:
feature_names=cv_model.get_feature_names_out()


In [51]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [52]:
tf_idf_vector=tfidf_transformer.transform(cv_model.transform(X_train))

results=[]
for i in range(tf_idf_vector.shape[0]):
    
    # get vector for a single document
    curr_vector=tf_idf_vector[i]
    
    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    
    results.append(keywords)

df=pd.DataFrame(zip(X_train,results),columns=['doc','keywords'])
df

,doc,keywords
0,william clark is a mixed race student in a cha...,"{'school': 0.29, 'fair': 0.252, 'clark': 0.189..."
1,by austin kocher syracuse university the last ...,"{'asylum': 0.531, 'mexico': 0.374, 'migrant': ..."
2,not even the most conspiratorial pandemic trut...,"{'stay home': 0.361, 'home': 0.277, 'salon': 0..."
3,washington — thousand of migrant child are bac...,"{'border': 0.405, 'child': 0.338, 'migrant': 0..."
4,when donald trump wa president he made no effo...,"{'politico': 0.371, 'harris': 0.283, 'governor..."
...,...,...
2147,greek church in thessaloniki have been accused...,"{'church': 0.775, 'coronavirus': 0.172, 'pasto..."
2148,during a sunday appearance on fox news sunday ...,"{'salon': 0.245, 'recovery': 0.216, 'shaped': ..."
2149,the federal bureau of land management ha a hab...,"{'land': 0.384, 'oil': 0.362, 'gas': 0.273, 'o..."
2150,with the debut of wandavison and the falcon an...,"{'black': 0.363, 'writer': 0.298, 'wa': 0.2, '..."


In [75]:
def search_keyword(df):
    keyword = 'biden'
    returned_articles = []
    for index, row in enumerate(df.keywords):
        if keyword in row:
            returned_articles.append((df['doc'][index],df['keywords'][index][keyword]))
    return returned_articles

In [76]:
search_keyword(df)

[('cnn president joe biden on sunday night lent his support to amazon worker who are pushing to unionize — and appeared to warn amazon amzn not to deter them in a video posted on twitter biden didn t mention the company by name but he did reference worker in alabama where a milestone union election is underway at an amazon facility in bessemer eligible worker at the facility are currently voting by mail to decide whether to form amazon s first u based union today and over the next few day and week worker in alabama and all across america are voting on whether to organize a union in their workplace biden said in the video there should be no intimidation no coercion no threat no anti union propaganda biden continued no supervisor should confront employee about their union preference you know every worker should have a free and fair choice to join a union the law guarantee that choice amazon did not immediately respond to a request for comment read more',
  0.182),
 ('during the campaign 

In [99]:
def search_keyword(df):
    keyword = 'biden'
    returned_articles = []
    for index, row in enumerate(df.keywords):
        if keyword in row:
            returned_articles.append((df['doc'][index],df['keywords'][index][keyword]))

    df = pd.DataFrame(returned_articles,columns=['text','keyword_score'])
    df = df.sort_values(by=['keyword_score'],ascending=False)
    return df[:10]

In [100]:
search_keyword(df)

,text,keyword_score
29,washington — joe biden never had a seat at the...,0.424
33,a new report from politico this week tried to ...,0.422
93,tara reade who claimed last year that joe bide...,0.371
79,when progressive activist melissa byrne wanted...,0.347
4,yesterday morning on a bike ride—one of those ...,0.344
11,young joe biden photo bettmann archive the dem...,0.340
13,for the love of god someone please help joe bi...,0.327
50,have an important tip let u know email u here ...,0.326
10,after a recent video showed president joe bide...,0.325
40,america s worst kept secret – and most terrify...,0.305


In [ ]:
# explaratory data analysis for each news source for presentation